In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "token_transfers"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="25309728:+1",
    # blockbatch/token_transfers/erc20_transfers_v1/chain=base/dt=2025-01-20/000025309600.parquet
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-02-03 09:40:55 [debug    ] connecting to GOLDSKY Clickhouse client... filename=client.py lineno=37 process=96674
2025-02-03 09:40:55 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=96674
2025-02-03 09:40:55 [debug    ] loaded vault: 18 items         filename=vault.py lineno=79 process=96674
2025-02-03 09:40:56 [debug    ] initialized GOLDSKY Clickhouse client. filename=client.py lineno=42 process=96674
2025-02-03 09:40:56 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=37 process=96674
2025-02-03 09:40:57 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=42 process=96674
2025-02-03 09:40:57 [info     ] prepared 1 input batches.      filename=byblock.py lineno=88 process=96674
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain',
                                                      value='base'),
                                      PartitionColumn(name='dt',
                   

In [2]:
logs_view = input_datasets["ingestion/logs_v1"].create_view()

transfers = auxiliary_templates["token_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": logs_view,
    },
)

2025-02-03 09:40:57 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=96674
2025-02-03 09:40:57 [info     ] created table/view ingestion_logs_v1_view filename=client.py lineno=202 process=96674
2025-02-03 09:40:57 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=96674
2025-02-03 09:40:57 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=96674 template=token_transfers


In [3]:
ctx.client.sql(f"""
    SELECT
        dt,
        chain,
        transaction_hash,
        array_length(l.indexed_args) as num_args,
        l.indexed_args,
        l.indexed_args[3] as token_id,
        hex_to_lossless(l.indexed_args[3]) AS token_id_losless
        
    
    FROM {logs_view} as l 
    WHERE l.block_number = 25309728
    AND l.topic0 = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
    AND l.transaction_hash = '0x01a2e1db82352c24db5e2bc931079e100c2a3bef82b008e205277d9a599ea2ce'
    -- where l.topic0 = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
    -- AND l.address = '0xf4dc475961095021a0fbfb17d336a6f0f13863af'
""")

┌────────────┬─────────┬────────────────────────────────────────────────────────────────────┬──────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬──────────────────┐
│     dt     │  chain  │                          transaction_hash                          │ num_args │                                                                                                 indexed_args                                                                                                 │                              token_id                              │ token_id_losless │
│    date    │ varchar │                              varchar                               │  int64   │                                                                                                

In [4]:
ctx.client.sql("""
    SELECT * FROM transfers as l 
    WHERE l.transaction_hash = '0x01a2e1db82352c24db5e2bc931079e100c2a3bef82b008e205277d9a599ea2ce'
""")

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────┬─────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │ amount │ amount_lossless │                from_address                │                 to_address                 │ token_id │
│    date    │ varchar │  int32   │ varchar │     uint32      │    int64     │                              varchar                               │         

In [5]:
print(input_datasets["ingestion/logs_v1"].as_subquery())

2025-02-03 09:41:02 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=96674
(

        SELECT * FROM 
        read_parquet(
            [
                'gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=base/dt=2025-01-20/000025309600.parquet'
            ],
            hive_partitioning = true
        )
        
        
        
)


In [6]:
erc20_transfers = (
    transfers
    .filter("token_id IS NULL")
    .project("* EXCLUDE token_id")
)

df = ctx.client.sql(f"SELECT * FROM erc20_transfers").df()
df.head()

,dt,chain,chain_id,network,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,contract_address,amount,amount_lossless,from_address,to_address
0,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x1f6ae0df0e094af81555d17689b1e063b189bdbcfa6a...,9,0,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,15000.0,15000,0xb361b42cff5e06acf3a7c1c45856ea6dbac1d87e,0xa227908e201a19df063608caea6b7fd01f7b93a4
1,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x7ec2b792172470bee82834e8d605d1131af1ed8bbc08...,10,1,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,1116.0,1116,0x4a63ab98add75f711fb2011976b0dcccc6f40a6c,0x3e70290021c7a6ec53b22fb38475358fd9152100
2,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x8953b900121e084b8ade6d9f73c4a6c7161228b7e368...,44,102,0xc31f8b70e03b1e66412b44b63268f5c9a0e7d2a0,NaN,2900000000000000000000,0xf0e795f51414da582c1ddf6ef82aa3df3c7bbf63,0x0000000000000000000000000000000000000000
3,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x8953b900121e084b8ade6d9f73c4a6c7161228b7e368...,44,104,0x8c9037d1ef5c6d1f6816278c7aaf5491d24cd527,NaN,27605606211677737712645,0x58708f65bacdf5040eac739cf01299fa756a1154,0x373065e66b32a1c428aa14698dfa99ba7199b55e
4,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x8953b900121e084b8ade6d9f73c4a6c7161228b7e368...,44,106,0x8c9037d1ef5c6d1f6816278c7aaf5491d24cd527,NaN,276056062116777377126,0x373065e66b32a1c428aa14698dfa99ba7199b55e,0xdc4e7e62f383e2a6f3071f7a9833c5ab7d4b239b


In [7]:
erc721_transfers = (
    transfers
    .filter("token_id IS NOT NULL")
    .project("* EXCLUDE (amount, amount_lossless)")
)
df = ctx.client.sql(f"SELECT * FROM erc721_transfers").df()
df.head()

,dt,chain,chain_id,network,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,contract_address,from_address,to_address,token_id
0,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x6053585370d674958378a6922e15ac3740576522b73b...,11,20,0x827922686190790b37229fd06084350e74485b72,0x60bdee58d18dd68af4a2ee39fe4e67def7f51dc2,0x0000000000000000000000000000000000000000,5904624
1,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0xb29941e01c12a12857919c1024b28f248a00d35b9339...,15,32,0x827922686190790b37229fd06084350e74485b72,0xbda319bc7cc8f0829df39ec0fff5d1e061ffadf7,0x54b854168dc92ace46914b2abb287d25d2b009fd,5904623
2,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x68f902a94d09aecd949a9926b2e9667aae7eb1c3453e...,127,338,0xd4ac69b84c6931aba6928668acb42e16f576ed5b,0x5b8faa3e012efc655a9e298510310fb07f2664e7,0xed45e4f56204c7f3bf07dc935c77f5d869ee7876,3296
3,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0xe1bd91e85a73cb93f4d78ea9f22af5c711385cfbb672...,132,341,0xe425183ce903a2387c00c35a6fb47fc6b9de8fd9,0x3432aacb8f4b324bc887087e0bb473ce4dfc64b8,0x0000000000000000000000000000000000000000,20000000951
4,2025-01-20,base,8453,mainnet,1737408547,25309600,0x33b103e4e25af5b283d491f8c9ebe222f1df92a3b6c4...,0x0eb238ea7f2625386c0bd47698200d20d90a57ebc23c...,155,368,0x7a56609c9d404f760cb2c45b72d04460a7823efb,0x0000000000000000000000000000000000000000,0x970b0efd250d8e2a039a208a4d3eebd81617e94c,94
